In [5]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

llm = ChatOpenAI(model_name="gpt-4-turbo-preview",
                 temperature=0.1,
                 )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)





In [6]:
examples = [
    {
        "question": "탑건",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are the world's most accurate film information expert. You must always respond with three emojis."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("괴물")
invoke_chain("모비우스")
invoke_chain("내가 가장 처음에 물어봤던 영화는 뭐였지?")

content='🇰🇷👹🌉'
content='🧛\u200d♂️🩸🌀'
content='괴물'
